In [ ]:
import numpy as np
import random
from scipy.stats import expon
import heapq
from collections import deque
from math import radians, cos, sin, asin, sqrt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import imageio
from tqdm.notebook import tqdm
from math import atan2, degrees

IMAGE_DIR = "simulation_images"  # Directory to store images
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

# Constants
WIDTH = (126.7642937, 127.1837947)  # Approximate longitude range for Seoul
HEIGHT = (37.4259627, 37.7017496)  # Approximate latitude range for Seoul
VELOCITY = 30  # ER van velocity (km/h), approx 8.33 m/s
UNIT_TIME = 1  # Set the unit time `u` as 1 second for simulation
LAMBDA = 1  # The lambda for exponential distribution, placeholder value
TREATMENT_LAMBDA = 0.1 # The lambda for exponential distribution, placeholder value
MAX_SIMULATION_TIME = 3600  # 1 hour of simulation time for example

# Data structures
# Data structures
hospitals = {
    'Seoul National University Hospital': {'location': (126.9991, 37.5796), 'queue': deque(), 'current_patient': None},
    'Asan Medical Center': {'location': (127.1079, 37.5262), 'queue': deque(), 'current_patient': None},
    'Samsung Medical Center': {'location': (127.0853, 37.4881), 'queue': deque(), 'current_patient': None},
    'Severance Hospital': {'location': (126.9408, 37.5622), 'queue': deque(), 'current_patient': None},
    'Korea University Guro Hospital': {'location': (126.8849, 37.4914), 'queue': deque(), 'current_patient': None},
    # ... Add additional hospitals as needed
}
patients = []
simulation_time = 0
DEATH_TOLL = 0
TREATED_PATIENTS = 0
TOTAL_ETAW = 0

def calculate_initial_bearing(pointA, pointB):
    # Calculate the initial bearing from point A to point B
    lon1, lat1, lon2, lat2 = map(radians, [pointA[0], pointA[1], pointB[0], pointB[1]])

    dlon = lon2 - lon1
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - (sin(lat1) * cos(lat2) * cos(dlon))

    initial_bearing = atan2(x, y)
    # Convert bearing from radians to degrees
    initial_bearing = degrees(initial_bearing)
    bearing = (initial_bearing + 360) % 360

    return bearing

def calculate_new_position(lat, lon, bearing, distance):
    """
    Calculate a new latitude and longitude point given a starting point,
    initial bearing, and distance traveled. This function assumes a spherical Earth.
    
    :param lat: Starting latitude in decimal degrees
    :param lon: Starting longitude in decimal degrees
    :param bearing: Bearing in decimal degrees
    :param distance: Distance traveled in meters
    :return: Tuple with new latitude and longitude in decimal degrees
    """
    
    # Earth radius in meters
    R = 6371000
    
    # Convert latitude, longitude, and bearing to radians
    lat_rad = radians(lat)
    lon_rad = radians(lon)
    bearing_rad = radians(bearing)
    
    # Calculate new latitude in radians
    new_lat_rad = sin(lat_rad) * cos(distance / R) + cos(lat_rad) * sin(distance / R) * cos(bearing_rad)
    
    # Calculate new longitude in radians
    new_lon_rad = lon_rad + atan2(sin(bearing_rad) * sin(distance / R) * cos(lat_rad),
                                   cos(distance / R) - sin(lat_rad) * sin(new_lat_rad))
    
    # Convert the new latitude and longitude from radians to degrees
    new_lat = degrees(new_lat_rad)
    new_lon = degrees(new_lon_rad)
    
    return new_lat, new_lon

def move_patients_towards_hospital():
    for _, patient in patients:
        if patient['status'] == 'transported':
            # Calculate the distance the patient moves during one unit of time
            distance_travelled = VELOCITY * 1000 / 3600 * UNIT_TIME  # in meters
            bearing = calculate_initial_bearing(patient['location'], hospitals[patient['hospital']]['location'])
            new_lat, new_lon = calculate_new_position(patient['location'][1], patient['location'][0], bearing, distance_travelled)
            patient['location'] = (new_lon, new_lat)

def generate_patient_location():
    longitude = random.uniform(*WIDTH)
    latitude = random.uniform(*HEIGHT)
    return (longitude, latitude)

def sample_exponential_time(lambda_value):
    return expon.rvs(scale=1/lambda_value)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def plot_simulation(simulation_time, patients, hospitals, IMAGE_DIR):
    fig, ax = plt.subplots()
    ax.set_xlim(WIDTH)
    ax.set_ylim(HEIGHT)
    ax.set_title(f'Simulation Time: {simulation_time} seconds')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    for hospital_id, hospital in hospitals.items():
        x, y = hospital['location']
        ax.plot(x, y, 'ro')  # Red 'o' for hospitals
        ax.text(x, y, hospital_id, fontsize=9)

    for patient_tuple in patients:
        patient = patient_tuple[1]  # Access the patient dictionary
        if patient['status'] == 'waiting':
            x, y = patient['location']
            ax.plot(x, y, 'bo')  # Blue 'o' for patients waiting

    plt.savefig(f"{IMAGE_DIR}/frame_{simulation_time:04d}.png")  # Save the frame
    plt.close(fig)  # Close the plot to avoid memory issues


def create_gif():
    images = []
    for file_name in sorted(os.listdir(IMAGE_DIR)):
        if file_name.endswith('.png'):
            file_path = os.path.join(IMAGE_DIR, file_name)
            images.append(imageio.imread(file_path))
    imageio.mimsave('simulation.gif', images, fps=60)  # fps controls the speed


# Replace the simple Euclidean distance calculation with the Haversine distance
def calculate_distance(patient_location, hospital_location):
    return haversine(patient_location[0], patient_location[1], hospital_location[0], hospital_location[1])

def calculate_etaw(distance, queue_length, lambda_value):
    travel_time = distance / (VELOCITY * 1000 / 3600)  # Correct velocity conversion to m/s
    waiting_time = lambda_value * (queue_length + 1 + np.sqrt(queue_length + 1))
    return travel_time + waiting_time

def determine_hospital(patient_location):
    etaws = {
        hospital_id: calculate_etaw(
            calculate_distance(patient_location, hospital['location']),
            len(hospital['queue']),
            LAMBDA
        ) for hospital_id, hospital in hospitals.items()
    }
    return min(etaws, key=etaws.get)

def update_hospital_queues():
    global TREATED_PATIENTS, DEATH_TOLL, TOTAL_ETAW
    # Move patients through queues
    for hospital_id, hospital in hospitals.items():
        if hospital['current_patient'] is not None:
            hospital['current_patient']['treatment_time'] -= UNIT_TIME
            if hospital['current_patient']['treatment_time'] <= 0:
                TREATED_PATIENTS += 1
                hospital['current_patient']['status'] = 'treated'  # Update the patient status
                hospital['current_patient'] = None
        if not hospital['current_patient'] and hospital['queue']:
            patient = hospital['queue'].popleft()
            hospital['current_patient'] = patient
            patient['status'] = 'in_treatment'  # Update the patient status
            patient['treatment_time'] = sample_exponential_time(TREATMENT_LAMBDA)  # Treatment time separate from arrival time



def run_simulation():
    global simulation_time, TOTAL_ETAW, DEATH_TOLL

    pbar = tqdm(total=MAX_SIMULATION_TIME)  # For visual feedback in notebooks
    
    while simulation_time < MAX_SIMULATION_TIME:
        # Simulate patient occurrence with correct time interval
        if not patients or simulation_time >= patients[0][1]['arrival_time']:
            patient_location = generate_patient_location()
            arrival_time = simulation_time + sample_exponential_time(LAMBDA)
            new_patient = {
                'location': patient_location,
                'arrival_time': arrival_time,
                'status': 'waiting'
            }
            # Push a tuple with arrival_time as the first element for sorting
            heapq.heappush(patients, (arrival_time, new_patient))

        # Update hospital queues
        update_hospital_queues()

        # Check for new patients
        while patients and patients[0][1]['arrival_time'] <= simulation_time:
            _, patient = heapq.heappop(patients)
            best_hospital = determine_hospital(patient['location'])
            patient['status'] = 'transported'
            patient['hospital'] = best_hospital  # Store the target hospital
            hospitals[best_hospital]['queue'].append(patient)
            # TOTAL_ETAW calculation would go here if needed

        # Visualize the simulation state
        plot_simulation(simulation_time, patients, hospitals, IMAGE_DIR)
        simulation_time += UNIT_TIME
        pbar.update(UNIT_TIME)  # Update progress bar
    
    pbar.close()  # Close the progress bar
    create_gif()  # Create a GIF after the simulation ends


# Example usage
run_simulation()  # This will run for the defined simulation time